In [ ]:
/* 
    Vấn đề:
        - Cấu hình store phức tạp
        - Phải thêm nhiều gói (thư viện/package)
        - Yêu cầu quá nhiều đoạn code sẵn
*/

In [ ]:
/* 
    - Redux-logger: Là thư viện giúp log ra 
        + Những action của redux
        + State trước/sau khi dispatch

    - Khi thực hiện một dispatch(action):
        1. Log ra [prev state]: Nhìn được state trước khi được action thực hiện
        2. Log ra [action]: Nhìn được action.type, action.payload
        3. Log ra [next state]: Nhìn được state sau khi được thực hiện. Next state chính là kết quả trả ra của reducer
*/


![alt text](https://retux.js.org/assets/img/redux-data-flow.ab0c08be.gif)

1. `Redux-Logger`

In [ ]:
Middlewares: Phần mềm trung gian: 
- Giống như là một người gác cổng với vai trò là KIỂM SOÁT:
    - Có quyền cho một người quay về/không cho vào
    - Có quyền cho một người đi qua
    - Có quyền trước khi vào thì khám xét
    - Có quyền lấy đi cái gì của người đi vào

- Giống như bạn vào công ty:
1. Bạn (client) đi qua bảo vệ (middleware).
2. Bảo vệ:
    - Kiểm tra thẻ
    - Ghi log bạn vào lúc mấy giờ
    - Xác minh bạn có được phép vào không
    - Sau đó mới cho bạn vào văn phòng (server).

- Tùy tác vụ, middleware có thể:
    - Kiểm tra quyền (authorization)
    - Xác thực (authentication)
    - Ghi log request
    - Chặn request nếu thấy bất thường
    - Sửa đổi request trước khi vào server
    - Bổ sung dữ liệu vào request
    - Xử lý lỗi

- TÓM LẠI:
- Nó giống như anh bảo vệ/anh gác cổng đứng giữa client (người gửi yêu cầu) và server (nơi xử lý yêu cầu).
    - Mọi request đi vào server → phải đi qua middleware.
    - Mọi response đi ra → cũng có thể đi qua middleware.


In [ ]:
Middlewares: Logger: Đứng ở giữa, khi dữ liệu đi đến => logger chỉ có vai trò đón dữ liệu đến để log ra thôi. Khi action đi đến:
1. Log state hiện tại
2. Log luôn action đó
3. Sau đó log luôn ra state được cập nhật

npm i redux-logger

In [ ]:
//src/store/index.jsx

import { applyMiddleware, legacy_createStore } from "redux"; // Bổ sung applyMiddleware
import logger from "redux-logger"; // Bổ sung
import rootReducer from "./rootReducer";

/* 
- Redux cung cấp sẵn một hàm là applyMiddleware
- Khi gọi applyMiddleware thì truyền middleware làm đối số: applyMiddleware(arg) => áp dụng đúng mô hình người gác cổng
- Tại createStore nhận tham số thứ 2 là applyMiddleware
*/

/* 
- createStore(rootReducer, preloadedState, enhancer):
Ban đầu hàm createStore nhận vào 3 tham số:
- rootReducer: reducer
- preloadedState: Giá trị khởi tạo (không bắt buộc)
- enhancer: Thêm middleware

? Vậy tại sao lại truyền applyMiddleware() là tham số thứ 2 mà không phải preloadedState:
=> Trong createStore sẽ kiểm tra:
- Nếu tham số thứ 2 không phải là hàm thì sẽ coi là preloadedState
- Nếu là hàm thì sẽ coi là enhancer (là tham số thứ 3)
- Chính applyMiddleware() gọi hàm sẽ trả ra một hàm khác
*/


const store = legacy_createStore(rootReducer, applyMiddleware(logger)); // Bổ sung tham số thứ 2 và truyền logger vào applyMiddleware: applyMiddleware(logger)
export { store };


In [ ]:
LƯU Ý: Không nên sử dụng logger ở trên Production:
- Nếu chậm thì cũng không lo:
    1. Đúng là mở tab Console thì nó log ra thì có chậm hơn thật
    2. Nhưng khi tắt DevTool thì nó đủ thông minh để không log ra => không ảnh hưởng đến hiệu năng
=> Nhưng chúng ta cũng nên để lộ ra Logic

In [ ]:
/* 
? Cách để trên Production không hiện Logger:
Nhận ra: Nếu không truyền vào trong applyMiddleware() => không có middleware  => Kiểm tra tham số thứ 3
*/

In [ ]:
// src/store/index.jsx


/* 1. Tạo mảng chứa các middleware */
const enhancers = [];

/* 3. Sử dụng các biến môi trường: Giúp biết được môi trường của chúng ta đang chạy ở đâu:
console.log(import.meta.env)
{
    BASE_URL: "/"
    DEV: true --> true: Đang ở Môi trường DEV
    MODE: "development"
    PROD: false --> Môi trường Production: (npm run build)
    SSR: false
}
*/

/* 4. Kiểm tra biến môi trường:
Nếu không phải là môi trường Product => thì mới dùng middleware logger */
if (!import.meta.env.PROD) {
    enhancers.push(logger);
}
/* 2. Trải các mảng vào đối số của hàm applyMiddleware */
const store = legacy_createStore(rootReducer, applyMiddleware(...enhancers));
export { store };


Kiểm tra trên môi trường Product:
1. npm run build
2. npm run preview